# 🏦 Predicția Riscului de Credit - Analiza Default-ului de Împrumut

## 📊 Descrierea Proiectului

**Problema Reală:** În domeniul financiar, băncile și instituțiile de creditare trebuie să evalueze riscul ca un client să nu își plătească împrumutul la timp (default). Această predicție ajută la:
- Minimizarea pierderilor financiare
- Stabilirea ratelor dobânzilor adecvate
- Luarea deciziilor de aprobare/respingere a împrumuturilor

**Obiectivul:** Construim un model de Machine Learning care să calculeze **scorul de propensiune** (probabilitatea) ca un client să intre în default.

---

## 🎯 Ce Vom Învăța

1. **Clasificare Binară** - Predicție "Default" (1) vs "No Default" (0)
2. **Modele Structurate:**
   - Regresia Logistică (interpretabilă, rapidă)
   - Linear Discriminant Analysis (LDA)
3. **K-Nearest Neighbors (KNN)** - Cu standardizare z-scores
4. **Evaluare pentru Date Imbalanced:**
   - Confusion Matrix
   - Precision, Recall, F1-Score
   - ROC Curve și AUC

---

## 📁 Dataset

Vom folosi **German Credit Data** de la UCI Machine Learning Repository - un dataset real folosit în industrie pentru training în credit scoring.

**Context:** 1000 de clienți cu 20 de atribute (vârstă, status cont bancar, credit history, ocupație, etc.) și rezultatul împrumutului.


## 1️⃣ Setup și Instalare Librării

In [ ]:
# Instalăm librării suplimentare dacă e necesar
!pip install ucimlrepo --quiet
!pip install imbalanced-learn --quiet

In [ ]:
# Importuri pentru manipularea datelor
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Vizualizare
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')

# Machine Learning - Preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# Machine Learning - Modele
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier

# Machine Learning - Evaluare
from sklearn.metrics import (
    confusion_matrix, classification_report, 
    roc_curve, roc_auc_score, auc,
    precision_recall_curve, average_precision_score,
    accuracy_score, precision_score, recall_score, f1_score
)

# Pentru date imbalanced
from imblearn.over_sampling import SMOTE

# Pentru accesarea dataset-ului
from ucimlrepo import fetch_ucirepo

print("✅ Toate librăriile au fost importate cu succes!")

## 2️⃣ Încărcarea Datelor Reale

In [ ]:
# Încărcăm German Credit Data de la UCI ML Repository
print("📥 Descărcăm German Credit Dataset de la UCI...")

# ID-ul pentru German Credit Data în UCI Repository
statlog_german_credit_data = fetch_ucirepo(id=144)

# Extragem features (X) și target (y)
X = statlog_german_credit_data.data.features
y = statlog_german_credit_data.data.targets

print(f"✅ Date încărcate: {X.shape[0]} clienți, {X.shape[1]} features")
print(f"\n📋 Primele coloane (înainte de redenumire):")
print(X.columns.tolist()[:5])

### 📝 Redenumirea Features-urilor

Dataset-ul vine cu atribute generice (Attribute1, Attribute2...). Le vom redenumi cu **numele lor reale** pentru claritate.

In [ ]:
# Dictionary cu maparea dintre Attribute1, Attribute2... și numele real
feature_names = {
    'Attribute1': 'checking_status',          # Status cont curent (A11: <0 DM, A12: 0-200 DM, A13: >200 DM, A14: no account)
    'Attribute2': 'duration_months',          # Durata împrumutului în luni
    'Attribute3': 'credit_history',           # Istoric credit (A30-A34: de la no credits la all paid back)
    'Attribute4': 'purpose',                  # Scopul împrumutului (car, furniture, radio/TV, etc.)
    'Attribute5': 'credit_amount',            # Suma împrumutului
    'Attribute6': 'savings',                  # Cont economii/bonds (A61: <100 DM, A62: 100-500, etc.)
    'Attribute7': 'employment_duration',      # Durata angajării actuale
    'Attribute8': 'installment_rate',         # Rata lunară ca % din venitul disponibil
    'Attribute9': 'personal_status_sex',      # Status personal și sex
    'Attribute10': 'other_debtors',           # Alți debitori/garanți
    'Attribute11': 'present_residence',       # Ani la reședința actuală
    'Attribute12': 'property',                # Tipul proprietății (real estate, savings, car, etc.)
    'Attribute13': 'age_years',               # Vârsta în ani
    'Attribute14': 'other_installments',      # Alte planuri de rate (bank, stores, none)
    'Attribute15': 'housing',                 # Tipul locuinței (rent, own, for free)
    'Attribute16': 'existing_credits',        # Număr de credite existente la această bancă
    'Attribute17': 'job',                     # Tipul jobului
    'Attribute18': 'num_dependents',          # Număr de persoane întreținute
    'Attribute19': 'telephone',               # Are telefon? (yes/no)
    'Attribute20': 'foreign_worker'           # Lucrător străin? (yes/no)
}

# Redenumim coloanele
X.rename(columns=feature_names, inplace=True)

print('✅ Features redenumite cu succes!\n')
print('📋 Lista completă de features:')
for i, col in enumerate(X.columns, 1):
    print(f'   {i:2d}. {col}')

print('\n💡 Features mai importante de observat:')
print('   🔹 credit_amount - Suma împrumutului (numeric)')
print('   🔹 duration_months - Durata împrumutului (numeric)')
print('   🔹 age_years - Vârsta clientului (numeric)')
print('   🔹 checking_status - Status cont curent (categorial)')
print('   🔹 credit_history - Istoricul de credit (categorial)')
print('   🔹 purpose - Scopul împrumutului (categorial)')

print('\n📊 Tipuri de date:')
print(X.dtypes.value_counts())

### 📖 Explicații Detaliate pentru Features Categoriale

**Features-urile categoriale au coduri specific (A11, A12, etc.). Iată ce înseamnă:**

#### 🏦 **checking_status** (Status cont curent):
- `A11`: < 0 DM (în descoperit)
- `A12`: 0 - 200 DM
- `A13`: >= 200 DM / salary assignments
- `A14`: Nu are cont curent

#### 📜 **credit_history** (Istoric credit):
- `A30`: No credits taken / all paid back
- `A31`: All credits at this bank paid back
- `A32`: Existing credits paid back duly till now
- `A33`: Delay in paying off in the past
- `A34`: Critical account / other existing credits

#### 🎯 **purpose** (Scopul împrumutului):
- `A40`: Car (new)
- `A41`: Car (used)
- `A42`: Furniture/equipment
- `A43`: Radio/television
- `A44`: Domestic appliances
- `A45`: Repairs
- `A46`: Education
- `A48`: Retraining
- `A49`: Business
- `A410`: Others

#### 💰 **savings** (Cont economii):
- `A61`: < 100 DM
- `A62`: 100 - 500 DM
- `A63`: 500 - 1000 DM
- `A64`: >= 1000 DM
- `A65`: Unknown / no savings account

#### 💼 **employment_duration**:
- `A71`: Unemployed
- `A72`: < 1 year
- `A73`: 1 - 4 years
- `A74`: 4 - 7 years
- `A75`: >= 7 years

*DM = Deutsche Mark (moneda Germaniei înainte de EURO)*

In [ ]:
# Arătăm câteva exemple de valori pentru features categoriale
print("📋 Exemple de valori pentru features categoriale:\n")

categorical_features = ['checking_status', 'credit_history', 'purpose', 'savings', 'employment_duration']

for feature in categorical_features:
    if feature in X.columns:
        print(f"\n🔹 {feature}:")
        print(X[feature].value_counts().head())
        print(f"   Total unique values: {X[feature].nunique()}")

## 3️⃣ Exploratory Data Analysis (EDA)

### 📊 Înțelegerea Datelor

In [ ]:
# Combinăm X și y pentru analiză
df = pd.concat([X, y], axis=1)

# Redenumim coloana target pentru claritate
df.rename(columns={'class': 'default'}, inplace=True)

print("🔍 Primele 5 rânduri din dataset:")
display(df.head())

print("\n📊 Informații despre tipurile de date:")
print(df.info())

print("\n📈 Statistici descriptive:")
display(df.describe())

### 🎯 Analiza Variabilei Target - Verificarea Imbalance-ului

**IMPORTANT:** În credit scoring, default-ul este o **clasă rară** (minority class). Acest lucru se numește **imbalanced data** și necesită atenție specială!

In [ ]:
# Transformăm target în 0/1 (0 = No Default, 1 = Default)
# În dataset-ul original: 1 = Good Credit, 2 = Bad Credit
df['default'] = df['default'].map({1: 0, 2: 1})  # 0 = No Default, 1 = Default

print("📊 Distribuția variabilei target:")
print(df['default'].value_counts())
print("\n📈 Proporții:")
print(df['default'].value_counts(normalize=True) * 100)

# Vizualizare
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Countplot
sns.countplot(data=df, x='default', ax=axes[0], palette=['#2ecc71', '#e74c3c'])
axes[0].set_title('Distribuția Default-ului', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Default (0 = Nu, 1 = Da)')
axes[0].set_ylabel('Număr de clienți')

# Pie chart
df['default'].value_counts().plot.pie(
    ax=axes[1], 
    autopct='%1.1f%%',
    labels=['No Default', 'Default'],
    colors=['#2ecc71', '#e74c3c'],
    startangle=90
)
axes[1].set_title('Proporția Default-ului', fontsize=14, fontweight='bold')
axes[1].set_ylabel('')

plt.tight_layout()
plt.show()

# Calculăm imbalance ratio
default_count = df['default'].sum()
no_default_count = len(df) - default_count
imbalance_ratio = no_default_count / default_count

print(f"\n⚠️ Imbalance Ratio: {imbalance_ratio:.2f}:1")
print(f"Pentru fiecare client cu default, avem {imbalance_ratio:.2f} clienți fără default.")
print("\n💡 Acest imbalance necesită tehnici speciale de evaluare!")

### 🔍 Analiza Features Importante

In [ ]:
# Selectăm features numerice pentru vizualizare
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_features.remove('default')

# Vizualizăm distribuția pentru primele 6 features numerice
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.ravel()

for idx, feature in enumerate(numeric_features[:6]):
    # Histograma separată pentru default vs no default
    df[df['default'] == 0][feature].hist(
        ax=axes[idx], bins=20, alpha=0.6, label='No Default', color='#2ecc71'
    )
    df[df['default'] == 1][feature].hist(
        ax=axes[idx], bins=20, alpha=0.6, label='Default', color='#e74c3c'
    )
    axes[idx].set_title(f'Distribuția: {feature}', fontsize=11, fontweight='bold')
    axes[idx].legend()
    axes[idx].set_xlabel(feature)
    axes[idx].set_ylabel('Frecvență')

plt.tight_layout()
plt.show()

print("\n💡 Observații:")
print("- Caută diferențe în distribuții între cele două clase")
print("- Features cu diferențe mari sunt predictori potențiali")

### 📊 Matricea de Corelație

In [ ]:
# Corelația pentru features numerice
numeric_df = df[numeric_features + ['default']]

plt.figure(figsize=(14, 12))
correlation_matrix = numeric_df.corr()

sns.heatmap(
    correlation_matrix, 
    annot=True, 
    fmt='.2f', 
    cmap='coolwarm',
    center=0,
    square=True,
    linewidths=1
)
plt.title('Matricea de Corelație - Features Numerice', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Cele mai corelate features cu target
print("\n🎯 Top features corelate cu DEFAULT (în valoare absolută):")
target_corr = correlation_matrix['default'].abs().sort_values(ascending=False)
print(target_corr[1:11])  # Excludem corelația cu el însuși

## 4️⃣ Preprocessing și Pregătirea Datelor

### 🔧 Ce Vom Face:
1. Tratarea valorilor lipsă (dacă există)
2. Encoding pentru variabile categoriale
3. Split Train/Test
4. **Standardizare (Z-scores)** - Crucial pentru KNN!

In [ ]:
# Verificăm valori lipsă
print("🔍 Verificăm valori lipsă:")
missing_values = df.isnull().sum()
if missing_values.sum() > 0:
    print(missing_values[missing_values > 0])
else:
    print("✅ Nu există valori lipsă!")

In [ ]:
# Encoding pentru variabile categoriale
print("🔄 Encoding variabile categoriale...")

# Identificăm coloanele categoriale
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

print(f"Coloane categoriale găsite: {len(categorical_cols)}")

# Label Encoding pentru categoriale
df_encoded = df.copy()
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
    label_encoders[col] = le
    print(f"  ✓ {col}: {len(le.classes_)} categorii")

print("\n✅ Encoding completat!")

In [ ]:
# Separăm features și target
X = df_encoded.drop('default', axis=1)
y = df_encoded['default']

print(f"📊 Shape features: {X.shape}")
print(f"📊 Shape target: {y.shape}")

# Train/Test Split cu stratificare (păstrăm proporția claselor)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,  # 20% pentru test
    random_state=42,
    stratify=y  # ⭐ IMPORTANT pentru date imbalanced!
)

print(f"\n📈 Train set: {X_train.shape[0]} samples")
print(f"📉 Test set: {X_test.shape[0]} samples")

# Verificăm proporțiile în train și test
print("\n🔍 Proporția default în Train:")
print(y_train.value_counts(normalize=True) * 100)
print("\n🔍 Proporția default în Test:")
print(y_test.value_counts(normalize=True) * 100)

### ⚖️ Standardizare (Z-Scores)

**De ce este crucială standardizarea?**

1. **Pentru KNN:** Distanța Euclideană este sensibilă la scalarea features:
   - Feature cu valori mari (ex: venit 10000-100000) va domina
   - Feature cu valori mici (ex: număr copii 0-5) va fi ignorat

2. **Z-Score Formula:** `z = (x - μ) / σ`
   - Transformă fiecare feature să aibă media 0 și deviație standard 1
   - Toate features ajung pe aceeași "scală"

3. **Pentru Logistic Regression și LDA:** Nu este strict necesară, dar ajută la convergență

In [ ]:
# Standardizare folosind StandardScaler
scaler = StandardScaler()

# ⚠️ IMPORTANT: Fit doar pe train, apoi transform pe ambele seturi!
# Aceasta previne "data leakage"
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convertim înapoi în DataFrame pentru clarity
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print("✅ Standardizare completată!")
print("\n📊 Verificăm statistici după standardizare (Train):")
print(f"Media: {X_train_scaled.mean().mean():.6f} (ar trebui ~0)")
print(f"Std Dev: {X_train_scaled.std().mean():.6f} (ar trebui ~1)")

# Vizualizăm efectul standardizării
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Înainte de standardizare
X_train.iloc[:, 0].hist(bins=30, ax=axes[0], color='#3498db', alpha=0.7)
axes[0].set_title(f'ÎNAINTE: {X_train.columns[0]}', fontweight='bold')
axes[0].set_xlabel('Valoare originală')
axes[0].set_ylabel('Frecvență')

# După standardizare
X_train_scaled.iloc[:, 0].hist(bins=30, ax=axes[1], color='#2ecc71', alpha=0.7)
axes[1].set_title(f'DUPĂ: {X_train.columns[0]} (Z-score)', fontweight='bold')
axes[1].set_xlabel('Z-score')
axes[1].set_ylabel('Frecvență')

plt.tight_layout()
plt.show()

### ⚖️ Tratarea Imbalance-ului cu SMOTE (Opțional)

**SMOTE** (Synthetic Minority Over-sampling Technique):
- Creează exemple sintetice pentru clasa minoritară (default)
- Funcționează prin interpolarea între vecini apropiați

**⚠️ Notă:** SMOTE se aplică DOAR pe train set!

In [ ]:
# Aplicăm SMOTE pentru a balansa clasele
print("🔄 Aplicăm SMOTE pentru balansarea claselor...")

smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)

print("\n📊 Distribuția ÎNAINTE de SMOTE:")
print(y_train.value_counts())
print("\n📊 Distribuția DUPĂ SMOTE:")
print(pd.Series(y_train_balanced).value_counts())

# Vizualizare comparație
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

pd.Series(y_train).value_counts().plot.bar(
    ax=axes[0], color=['#2ecc71', '#e74c3c'], alpha=0.7
)
axes[0].set_title('ÎNAINTE de SMOTE', fontweight='bold')
axes[0].set_xlabel('Clasă')
axes[0].set_ylabel('Număr de exemple')

pd.Series(y_train_balanced).value_counts().plot.bar(
    ax=axes[1], color=['#2ecc71', '#e74c3c'], alpha=0.7
)
axes[1].set_title('DUPĂ SMOTE', fontweight='bold')
axes[1].set_xlabel('Clasă')
axes[1].set_ylabel('Număr de exemple')

plt.tight_layout()
plt.show()

print("\n✅ SMOTE aplicat cu succes!")
print("\n💡 Vom antrena modele pe AMBELE seturi (original și SMOTE) pentru comparație.")

## 5️⃣ Construirea Modelelor de Machine Learning

### 📚 Modelele Pe Care Le Vom Antrena:

1. **Logistic Regression**
   - Model liniar pentru clasificare binară
   - Returnează probabilități între 0 și 1
   - Avantaje: Rapid, interpretabil (coeficienți explicabili)

2. **Linear Discriminant Analysis (LDA)**
   - Găsește combinația liniară de features care separă cel mai bine clasele
   - Presupune distribuție normală a datelor
   - Avantaje: Reducere de dimensionalitate built-in

3. **K-Nearest Neighbors (KNN)**
   - Clasifică bazat pe "vecinii" apropiați în spațiul features
   - Folosește distanța Euclideană (de aceea standardizarea e crucială!)
   - Avantaje: Non-parametric, flexibil

---

### 1️⃣ Logistic Regression

In [ ]:
print("🔵 Antrenăm Logistic Regression...\n")

# Model 1: Pe date originale (imbalanced)
log_reg_original = LogisticRegression(
    random_state=42,
    max_iter=1000,
    class_weight='balanced'  # Ajustare automată pentru imbalance
)
log_reg_original.fit(X_train_scaled, y_train)

# Model 2: Pe date SMOTE (balanced)
log_reg_smote = LogisticRegression(
    random_state=42,
    max_iter=1000
)
log_reg_smote.fit(X_train_balanced, y_train_balanced)

# Predicții
y_pred_log_orig = log_reg_original.predict(X_test_scaled)
y_pred_log_smote = log_reg_smote.predict(X_test_scaled)

# Probabilități (pentru AUC)
y_proba_log_orig = log_reg_original.predict_proba(X_test_scaled)[:, 1]
y_proba_log_smote = log_reg_smote.predict_proba(X_test_scaled)[:, 1]

print("✅ Logistic Regression antrenat!")

# Vizualizăm coeficienții (interpretabilitate)
coefficients = pd.DataFrame({
    'Feature': X_train.columns,
    'Coeficient': log_reg_original.coef_[0]
}).sort_values('Coeficient', key=abs, ascending=False).head(15)

plt.figure(figsize=(12, 8))
colors = ['#e74c3c' if x < 0 else '#2ecc71' for x in coefficients['Coeficient']]
plt.barh(coefficients['Feature'], coefficients['Coeficient'], color=colors)
plt.xlabel('Coeficient (Impact asupra probabilității de default)', fontsize=11)
plt.title('Top 15 Features - Logistic Regression (Interpretabilitate)', fontweight='bold', fontsize=13)
plt.axvline(x=0, color='black', linestyle='--', linewidth=2)
plt.tight_layout()
plt.show()

print("\n📊 Interpretarea Coeficienților:\n")
print("💡 Coeficienți POZITIVI (🟢) = Cresc probabilitatea de DEFAULT")
print("   Exemplu: Dacă 'duration_months' are coeficient pozitiv, înseamnă că")
print("   împrumuturi pe termen mai lung → risc mai mare de default\n")

print("💡 Coeficienți NEGATIVI (🔴) = Scad probabilitatea de DEFAULT")
print("   Exemplu: Dacă 'checking_status' are coeficient negativ, înseamnă că")
print("   conturi cu sold mai mare → risc mai mic de default\n")

print("🔝 Top 5 factori de risc (valoare absolută):")
for idx, row in coefficients.head(5).iterrows():
    direction = "⬆️ CREȘTE" if row['Coeficient'] > 0 else "⬇️ SCADE"
    print(f"   {direction} riscul: {row['Feature']} (coef: {row['Coeficient']:.4f})")

### 2️⃣ Linear Discriminant Analysis (LDA)

In [ ]:
print("🔵 Antrenăm Linear Discriminant Analysis (LDA)...\n")

# Model 1: Pe date originale
lda_original = LinearDiscriminantAnalysis()
lda_original.fit(X_train_scaled, y_train)

# Model 2: Pe date SMOTE
lda_smote = LinearDiscriminantAnalysis()
lda_smote.fit(X_train_balanced, y_train_balanced)

# Predicții
y_pred_lda_orig = lda_original.predict(X_test_scaled)
y_pred_lda_smote = lda_smote.predict(X_test_scaled)

# Probabilități
y_proba_lda_orig = lda_original.predict_proba(X_test_scaled)[:, 1]
y_proba_lda_smote = lda_smote.predict_proba(X_test_scaled)[:, 1]

print("✅ LDA antrenat!")
print(f"\n📊 Explained variance ratio: {lda_original.explained_variance_ratio_[0]:.4f}")

### 3️⃣ K-Nearest Neighbors (KNN)

**Cum funcționează KNN:**
1. Pentru o nouă observație, găsește cele mai apropiate K puncte din train set
2. Clasifică bazat pe "votul" majoritar al vecinilor
3. Distanța este Euclideană: `d = √Σ(x₁ - x₂)²`

**⚠️ De aceea standardizarea este CRUCIALĂ pentru KNN!**

In [ ]:
# Găsim cel mai bun K folosind cross-validation
print("🔍 Căutăm cel mai bun K pentru KNN...\n")

k_range = range(1, 31, 2)  # Testăm K-uri impare de la 1 la 29
cv_scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
    scores = cross_val_score(
        knn, X_train_scaled, y_train, 
        cv=5, 
        scoring='roc_auc'  # Folosim AUC pentru evaluare
    )
    cv_scores.append(scores.mean())

# Găsim best K
best_k = k_range[np.argmax(cv_scores)]
print(f"✅ Cel mai bun K: {best_k} (AUC = {max(cv_scores):.4f})")

# Vizualizare
plt.figure(figsize=(10, 6))
plt.plot(k_range, cv_scores, marker='o', linestyle='-', color='#3498db', linewidth=2)
plt.xlabel('Numărul de Vecini (K)', fontsize=12)
plt.ylabel('Cross-Validation AUC Score', fontsize=12)
plt.title('Găsirea Optimalului K pentru KNN', fontsize=14, fontweight='bold')
plt.axvline(x=best_k, color='#e74c3c', linestyle='--', label=f'Best K = {best_k}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
print(f"\n🔵 Antrenăm KNN cu K={best_k}...\n")

# Model 1: Pe date originale
knn_original = KNeighborsClassifier(
    n_neighbors=best_k,
    metric='euclidean',
    weights='distance'  # Vecini mai apropiați au greutate mai mare
)
knn_original.fit(X_train_scaled, y_train)

# Model 2: Pe date SMOTE
knn_smote = KNeighborsClassifier(
    n_neighbors=best_k,
    metric='euclidean',
    weights='distance'
)
knn_smote.fit(X_train_balanced, y_train_balanced)

# Predicții
y_pred_knn_orig = knn_original.predict(X_test_scaled)
y_pred_knn_smote = knn_smote.predict(X_test_scaled)

# Probabilități
y_proba_knn_orig = knn_original.predict_proba(X_test_scaled)[:, 1]
y_proba_knn_smote = knn_smote.predict_proba(X_test_scaled)[:, 1]

print("✅ KNN antrenat!")

## 6️⃣ Evaluarea Modelelor

### 📊 Metrici pentru Date Imbalanced

**De ce acuratețea NU este suficientă?**

Exemplu: Dacă 70% din clienți NU sunt în default:
- Un model "prost" care prezice ÎNTOTDEAUNA "no default" → 70% acuratețe!
- Dar ratează TOATE cazurile de default (cele pe care vrem să le prindem!)

**Metrici importante:**

1. **Confusion Matrix:**
   ```
                  Predicted
                0         1
   Actual  0    TN        FP
           1    FN        TP
   ```

2. **Precision:** TP / (TP + FP) - Din ce am prezis "default", câte sunt cu adevărat?

3. **Recall (Sensitivity):** TP / (TP + FN) - Din toți care sunt "default", pe câți i-am prins?

4. **F1-Score:** Media armonică dintre Precision și Recall

5. **AUC (Area Under ROC Curve):** Măsoară capacitatea de separare a claselor (0.5 = random, 1.0 = perfect)

---

In [ ]:
def evaluate_model(y_true, y_pred, y_proba, model_name):
    """
    Funcție completă de evaluare pentru date imbalanced
    """
    print(f"\n{'='*60}")
    print(f"📊 EVALUARE MODEL: {model_name}")
    print(f"{'='*60}\n")
    
    # Metrici de bază
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_proba)
    
    print("📈 METRICI:")
    print(f"   Accuracy:  {accuracy:.4f}")
    print(f"   Precision: {precision:.4f} ⭐")
    print(f"   Recall:    {recall:.4f} ⭐")
    print(f"   F1-Score:  {f1:.4f}")
    print(f"   AUC:       {auc:.4f} ⭐⭐⭐\n")
    
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Confusion Matrix heatmap
    sns.heatmap(
        cm, annot=True, fmt='d', cmap='Blues', 
        xticklabels=['No Default', 'Default'],
        yticklabels=['No Default', 'Default'],
        ax=axes[0]
    )
    axes[0].set_title(f'Confusion Matrix - {model_name}', fontweight='bold')
    axes[0].set_ylabel('Actual')
    axes[0].set_xlabel('Predicted')
    
    # ROC Curve
    fpr, tpr, _ = roc_curve(y_true, y_proba)
    axes[1].plot(fpr, tpr, color='#e74c3c', lw=2, label=f'AUC = {auc:.4f}')
    axes[1].plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--', label='Random')
    axes[1].set_xlabel('False Positive Rate')
    axes[1].set_ylabel('True Positive Rate (Recall)')
    axes[1].set_title(f'ROC Curve - {model_name}', fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Classification Report detaliat
    print("\n📋 CLASSIFICATION REPORT:")
    print(classification_report(
        y_true, y_pred, 
        target_names=['No Default', 'Default']
    ))
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc
    }

### 📊 Evaluarea Tuturor Modelelor

In [ ]:
# Evaluăm Logistic Regression
results_log_orig = evaluate_model(
    y_test, y_pred_log_orig, y_proba_log_orig,
    "Logistic Regression (Original Data)"
)

results_log_smote = evaluate_model(
    y_test, y_pred_log_smote, y_proba_log_smote,
    "Logistic Regression (SMOTE)"
)

In [ ]:
# Evaluăm LDA
results_lda_orig = evaluate_model(
    y_test, y_pred_lda_orig, y_proba_lda_orig,
    "Linear Discriminant Analysis (Original Data)"
)

results_lda_smote = evaluate_model(
    y_test, y_pred_lda_smote, y_proba_lda_smote,
    "Linear Discriminant Analysis (SMOTE)"
)

In [ ]:
# Evaluăm KNN
results_knn_orig = evaluate_model(
    y_test, y_pred_knn_orig, y_proba_knn_orig,
    f"K-Nearest Neighbors (K={best_k}, Original Data)"
)

results_knn_smote = evaluate_model(
    y_test, y_pred_knn_smote, y_proba_knn_smote,
    f"K-Nearest Neighbors (K={best_k}, SMOTE)"
)

## 7️⃣ Compararea Modelelor

### 🏆 Care model este cel mai bun?

In [ ]:
# Creăm DataFrame cu rezultatele
comparison_df = pd.DataFrame({
    'Model': [
        'Logistic Reg (Original)',
        'Logistic Reg (SMOTE)',
        'LDA (Original)',
        'LDA (SMOTE)',
        'KNN (Original)',
        'KNN (SMOTE)'
    ],
    'Accuracy': [
        results_log_orig['accuracy'],
        results_log_smote['accuracy'],
        results_lda_orig['accuracy'],
        results_lda_smote['accuracy'],
        results_knn_orig['accuracy'],
        results_knn_smote['accuracy']
    ],
    'Precision': [
        results_log_orig['precision'],
        results_log_smote['precision'],
        results_lda_orig['precision'],
        results_lda_smote['precision'],
        results_knn_orig['precision'],
        results_knn_smote['precision']
    ],
    'Recall': [
        results_log_orig['recall'],
        results_log_smote['recall'],
        results_lda_orig['recall'],
        results_lda_smote['recall'],
        results_knn_orig['recall'],
        results_knn_smote['recall']
    ],
    'F1-Score': [
        results_log_orig['f1'],
        results_log_smote['f1'],
        results_lda_orig['f1'],
        results_lda_smote['f1'],
        results_knn_orig['f1'],
        results_knn_smote['f1']
    ],
    'AUC': [
        results_log_orig['auc'],
        results_log_smote['auc'],
        results_lda_orig['auc'],
        results_lda_smote['auc'],
        results_knn_orig['auc'],
        results_knn_smote['auc']
    ]
})

print("\n" + "="*80)
print("🏆 COMPARAȚIA FINALĂ A MODELELOR")
print("="*80 + "\n")
display(comparison_df.style.highlight_max(axis=0, subset=['Precision', 'Recall', 'F1-Score', 'AUC'], color='lightgreen'))

# Vizualizare comparativă
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics = ['Precision', 'Recall', 'F1-Score', 'AUC']
colors = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12', '#9b59b6', '#1abc9c']

for idx, metric in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    comparison_df.plot(
        x='Model', y=metric, kind='bar', 
        ax=ax, color=colors, legend=False
    )
    ax.set_title(f'Comparație: {metric}', fontsize=13, fontweight='bold')
    ax.set_xlabel('')
    ax.set_ylabel(metric)
    ax.tick_params(axis='x', rotation=45)
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Identificăm cel mai bun model
best_model_idx = comparison_df['AUC'].idxmax()
best_model_name = comparison_df.loc[best_model_idx, 'Model']
best_model_auc = comparison_df.loc[best_model_idx, 'AUC']

print(f"\n🏆 CEL MAI BUN MODEL (bazat pe AUC): {best_model_name}")
print(f"   AUC Score: {best_model_auc:.4f}")
print(f"   Precision: {comparison_df.loc[best_model_idx, 'Precision']:.4f}")
print(f"   Recall: {comparison_df.loc[best_model_idx, 'Recall']:.4f}")

### 📈 Compararea ROC Curves pentru Toate Modelele

In [ ]:
# Plot toate ROC curves pe același grafic
plt.figure(figsize=(12, 8))

models_data = [
    (y_proba_log_orig, 'Logistic Reg (Original)', '#3498db'),
    (y_proba_log_smote, 'Logistic Reg (SMOTE)', '#2ecc71'),
    (y_proba_lda_orig, 'LDA (Original)', '#e74c3c'),
    (y_proba_lda_smote, 'LDA (SMOTE)', '#f39c12'),
    (y_proba_knn_orig, 'KNN (Original)', '#9b59b6'),
    (y_proba_knn_smote, 'KNN (SMOTE)', '#1abc9c')
]

for y_proba, name, color in models_data:
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    auc_score = roc_auc_score(y_test, y_proba)
    plt.plot(fpr, tpr, label=f'{name} (AUC = {auc_score:.4f})', 
             color=color, lw=2)

plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate (Recall)', fontsize=12)
plt.title('Comparația ROC Curves - Toate Modelele', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 Interpretare:")
print("   - Curba mai aproape de colțul stânga-sus = model mai bun")
print("   - AUC = 0.5 → model random (linie diagonală)")
print("   - AUC = 1.0 → model perfect")
print("   - AUC > 0.7 → model considerat bun în practică")

## 8️⃣ Aplicații Practice și Impact Business

### 💼 Cum se folosește acest model în realitate?

In [ ]:
# Simulăm scoring pentru clienți noi
print("🎯 SIMULARE: Scoring clienți noi\n")
print("="*60)

# Luăm 5 exemple random din test set
sample_indices = np.random.choice(X_test_scaled.index, 5, replace=False)
sample_data = X_test_scaled.loc[sample_indices]
sample_labels = y_test.loc[sample_indices]

# Folosim cel mai bun model pentru predicție
# Presupunem că Logistic Regression (Original) este cel mai bun
best_model = log_reg_original

for idx, (sample_idx, sample) in enumerate(sample_data.iterrows(), 1):
    # Predicție probabilitate
    proba = best_model.predict_proba(sample.values.reshape(1, -1))[0, 1]
    prediction = best_model.predict(sample.values.reshape(1, -1))[0]
    actual = sample_labels.loc[sample_idx]
    
    print(f"\n👤 CLIENT #{idx}")
    print(f"   Risc de Default: {proba*100:.2f}%")
    print(f"   Predicție: {'❌ DEFAULT' if prediction == 1 else '✅ NO DEFAULT'}")
    print(f"   Realitate: {'❌ DEFAULT' if actual == 1 else '✅ NO DEFAULT'}")
    print(f"   Status: {'✅ CORECT' if prediction == actual else '❌ GREȘIT'}")
    
    # Decizie de business
    if proba < 0.3:
        risk_level = "🟢 RISC SCĂZUT"
        action = "Aprobare automată, rată dobândă standard"
    elif proba < 0.6:
        risk_level = "🟡 RISC MEDIU"
        action = "Revizie manuală, posibil rată dobândă crescută"
    else:
        risk_level = "🔴 RISC RIDICAT"
        action = "Respingere sau condiții stricte + garanții"
    
    print(f"   {risk_level}")
    print(f"   📋 Acțiune recomandată: {action}")
    print("   " + "-"*55)

print("\n" + "="*60)

### ⚖️ Ajustarea Threshold-ului de Decizie

**Context Business:**
- **Threshold înalt (ex: 0.7):** Mai puține false positives → Mai mulți clienți aprobați, dar risc mai mare
- **Threshold scăzut (ex: 0.3):** Mai puține false negatives → Mai conservativ, dar pierdem clienți buni

Alegerea threshold-ului depinde de **costurile de business**!

In [ ]:
# Analizăm diferite threshold-uri
thresholds = np.arange(0.1, 1.0, 0.05)
precisions = []
recalls = []
f1_scores = []

for threshold in thresholds:
    y_pred_thresh = (y_proba_log_orig >= threshold).astype(int)
    precisions.append(precision_score(y_test, y_pred_thresh, zero_division=0))
    recalls.append(recall_score(y_test, y_pred_thresh, zero_division=0))
    f1_scores.append(f1_score(y_test, y_pred_thresh, zero_division=0))

# Vizualizare
plt.figure(figsize=(12, 6))
plt.plot(thresholds, precisions, 'o-', label='Precision', color='#3498db', linewidth=2)
plt.plot(thresholds, recalls, 's-', label='Recall', color='#e74c3c', linewidth=2)
plt.plot(thresholds, f1_scores, '^-', label='F1-Score', color='#2ecc71', linewidth=2)

# Găsim threshold optim (maxim F1)
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]
plt.axvline(x=optimal_threshold, color='black', linestyle='--', 
            label=f'Optimal Threshold = {optimal_threshold:.2f}')

plt.xlabel('Decision Threshold', fontsize=12)
plt.ylabel('Score', fontsize=12)
plt.title('Impactul Threshold-ului asupra Metricilor', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n🎯 Threshold Optim (maxim F1): {optimal_threshold:.2f}")
print(f"   Precision la acest threshold: {precisions[optimal_idx]:.4f}")
print(f"   Recall la acest threshold: {recalls[optimal_idx]:.4f}")
print(f"   F1-Score la acest threshold: {f1_scores[optimal_idx]:.4f}")

## 9️⃣ Concluzii și Pași Următori

### ✅ Ce Am Învățat

1. **Clasificare Binară pentru Credit Scoring**
   - Am construit modele care prezic probabilitatea de default
   - Am folosit date reale din industrie (German Credit Data)

2. **Tehnici de Machine Learning**
   - **Logistic Regression:** Rapid și interpretabil
   - **LDA:** Bun pentru separarea claselor cu distribuții normale
   - **KNN:** Flexibil, dar necesită standardizare (z-scores)

3. **Tratarea Date Imbalanced**
   - SMOTE pentru balansarea claselor
   - Metrici speciale: Precision, Recall, F1, AUC
   - Confusion Matrix pentru înțelegerea erorilor

4. **Importanța Standardizării**
   - Z-scores pentru KNN (distanță Euclideană)
   - Prevenirea dominației features cu scale mari

### 🚀 Pași Următori (Îmbunătățiri Posibile)

1. **Feature Engineering:**
   - Crearea de noi features (ratios, interactions)
   - Selecția features-urilor cele mai importante

2. **Modele Mai Avansate:**
   - Random Forest
   - Gradient Boosting (XGBoost, LightGBM)
   - Neural Networks

3. **Optimizare Hyperparameters:**
   - Grid Search / Random Search
   - Cross-validation mai robust (5-fold, 10-fold)

4. **Cost-Sensitive Learning:**
   - Definirea costurilor pentru FP vs FN
   - Optimizarea pentru profit maxim, nu doar acuratețe

5. **Model Deployment:**
   - Salvarea modelului (pickle, joblib)
   - API pentru scoring în timp real
   - Monitoring performanța în producție

### 📚 Resurse Suplimentare

- [Scikit-learn Documentation](https://scikit-learn.org/)
- [Imbalanced-learn](https://imbalanced-learn.org/)
- [Credit Scoring Best Practices](https://www.kaggle.com/learn/credit-risk-model-stability)

---

### 💡 Mesaj Final

Acest notebook demonstrează un **workflow complet** pentru un proiect real de Data Science:
1. ✅ Date reale (UCI Repository)
2. ✅ EDA comprehensiv
3. ✅ Preprocessing corect (stratification, standardization)
4. ✅ Multiple modele (Logistic, LDA, KNN)
5. ✅ Evaluare riguroasă pentru date imbalanced
6. ✅ Interpretare business

**Acesta este un proiect portfolio-ready pentru poziții entry-level în Data Science!** 🎯

---

*Creat pentru învățare și practică în Machine Learning aplicat în domeniul financiar.*


## 🔟 Bonus: Salvarea și Încărcarea Modelului

In [ ]:
import joblib

# Salvăm cel mai bun model
model_filename = 'credit_risk_model.pkl'
scaler_filename = 'credit_risk_scaler.pkl'

joblib.dump(log_reg_original, model_filename)
joblib.dump(scaler, scaler_filename)

print(f"✅ Model salvat: {model_filename}")
print(f"✅ Scaler salvat: {scaler_filename}")

# Pentru a încărca modelul ulterior:
# loaded_model = joblib.load(model_filename)
# loaded_scaler = joblib.load(scaler_filename)

print("\n💾 Pentru deploy, ai nevoie de:")
print("   1. Fișierul cu modelul (.pkl)")
print("   2. Fișierul cu scaler-ul (.pkl)")
print("   3. Lista de features în ordinea corectă")
print("   4. Pipeline de preprocessing (encoding)")